In [1]:
!pip install mahaNLP==0.5 gdown -q

In [2]:
!gdown --id 1VDwcsXYmh1uq8X7-8va25CP1JqL4xdv0

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1VDwcsXYmh1uq8X7-8va25CP1JqL4xdv0
To: /kaggle/working/mahasent_train_new_part1.csv
100%|██████████████████████████████████████| 3.23M/3.23M [00:00<00:00, 75.6MB/s]


In [3]:
# importing the necessary libraries
import mahaNLP
import pandas as pd
import torch
import random
import re
import gdown

In [4]:
from mahaNLP.tokenizer import Tokenize
tokenizer = Tokenize()
from mahaNLP.tagger import EntityRecognizer
ner_tagger = EntityRecognizer()
from mahaNLP.maskFill import MaskPredictor
mask_predictor = MaskPredictor()

In [5]:
import re

# preprocessing function, don't remove the punctuations at all.
def remove_non_marathi_in_tweets(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
#     i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [6]:
df = pd.read_csv('mahasent_train_new_part1.csv')
df.head()

,Unnamed: 0,tweet,label
0,0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे वृत्त दु:खद आहे. चार दशकं त्यांनी आपल्या परखड लेखणीने पत्रकारितेत अमूल्य योगदान दिले. दीक्षित यांच्या मार्गदर्शनाखाली पत्रकारांची पिढी घडली. अनंत दीक्षित यांना भावपूर्ण श्रद्धांजली! \nआम्ही त्यांच्या परिवाराच्या दु:खात सहभागी आहोत. pic.twitter.com/s3gnQQLtpk\n,-1
1,1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिसांनी प्रा. आनंद तेलतुंबडेंना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा-या पोलीस अधिका-यांवर तात्काळ कारवाई केली पाहिजे.\n#AnandTeltumbde\n,-1
2,2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांचा अवमान व विश्वासघात केला आहे फसवणूक झालेल्या शिवसैनिकांबद्दल मला सहानुभूती असून स्वार्थासाठी उद्धव ठाकरे भाजपसोबत गेले तरी मोदींचे सरकार घालवण्यातच महाराष्ट्राचे हित आहे हे जाणणारे शिवसैनिक मोदींच्या विरोधात काँग्रेसला मतदान करतील\n,-1
3,3,"आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भूत आहेत. यातील दिव्यांग हा एक घटक, जो शारीरिकदृष्ट्या कमकुवत आहे, त्याच्याकडे समाजाचे दुर्लक्ष होतेय. त्यामुळे सरकारने अशा व्यक्तींसाठी उपक्रम राबवून त्यांना समाजात सन्मानाने जगता येईल, याकडे लक्ष देण्याची गरज आहे. pic.twitter.com/g0tbJyyTPX\n",1
4,4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म्हणणारे सावरकर आम्हाला मान्य नाहीत. द्वेष आणि तिरस्कार हिंदू धर्मातही अभिप्रेत कधीच नव्हता. आम्ही सावरकरांच्या विचारांचा विरोध करताच राहू त्यात व्यक्तीद्वेष नाही तर वैचारिक विरोध आहे. हा देश गांधींच्या विचारांनीच चालेल!\n,-1


In [7]:
df.drop(['Unnamed: 0'], inplace=True, axis=1)

In [8]:
for i in range(len(df)):
  df['tweet'][i] = remove_non_marathi_in_tweets(df['tweet'][i])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df = df[df['tweet']!='']

In [10]:
df.head()

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे वृत्त दु:खद आहे. चार दशकं त्यांनी आपल्या परखड लेखणीने पत्रकारितेत अमूल्य योगदान दिले. दीक्षित यांच्या मार्गदर्शनाखाली पत्रकारांची पिढी घडली. अनंत दीक्षित यांना भावपूर्ण श्रद्धांजली! आम्ही त्यांच्या परिवाराच्या दु:खात सहभागी आहोत.,-1
1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिसांनी प्रा. आनंद तेलतुंबडेंना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा-या पोलीस अधिका-यांवर तात्काळ कारवाई केली पाहिजे.,-1
2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांचा अवमान व विश्वासघात केला आहे फसवणूक झालेल्या शिवसैनिकांबद्दल मला सहानुभूती असून स्वार्थासाठी उद्धव ठाकरे भाजपसोबत गेले तरी मोदींचे सरकार घालवण्यातच महाराष्ट्राचे हित आहे हे जाणणारे शिवसैनिक मोदींच्या विरोधात काँग्रेसला मतदान करतील,-1
3,"आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भूत आहेत. यातील दिव्यांग हा एक घटक, जो शारीरिकदृष्ट्या कमकुवत आहे, त्याच्याकडे समाजाचे दुर्लक्ष होतेय. त्यामुळे सरकारने अशा व्यक्तींसाठी उपक्रम राबवून त्यांना समाजात सन्मानाने जगता येईल, याकडे लक्ष देण्याची गरज आहे.",1
4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म्हणणारे सावरकर आम्हाला मान्य नाहीत. द्वेष आणि तिरस्कार हिंदू धर्मातही अभिप्रेत कधीच नव्हता. आम्ही सावरकरांच्या विचारांचा विरोध करताच राहू त्यात व्यक्तीद्वेष नाही तर वैचारिक विरोध आहे. हा देश गांधींच्या विचारांनीच चालेल!,-1


In [11]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [12]:
len(df)

6045

In [13]:
def mask_NER_entities_in_text_sequential(text):
    # getting the NER tokens from the sentence
    df_ner = ner_tagger.getTokenLabels(text)

    # converting the NER tokens into a list, excluding those from the 'Other' category
    ner_tokens = list(df_ner.loc[df_ner['entity_group'] != 'Other', 'word'])
    # print('ner_tokens', ner_tokens)
    
    for token in ner_tokens:
      text = text.replace(token, token+" ", 1)
    
    for token in ner_tokens:
      if token not in text:
        ner_tokens.remove(token)


    sentence=text
    for i in ner_tokens:
        # replacing the first occurance of the NER token with [MASK]
        temp = sentence.replace(i, '[MASK] ', 1)

        # getting the prediction with the highest score, and replacing the sentence with that
        if(mask_predictor.predictMask(temp)['token_str'][0]!=i):
          sentence = mask_predictor.predictMask(temp)['sequence'][0]
        else:
          sentence = mask_predictor.predictMask(temp)['sequence'][1]

    return(sentence)

In [14]:
df['tweet'][69]

'आधुनिक भारताच्या विकासाचा पाया ज्यांच्या विचारावर उभा आहे ते देशाचे प्रथम पंतप्रधान पंडित जवाहरलाल नेहरु यांचा आज जन्मदिन. केवळ राजकारणच नव्हे तर एक तत्वचिंतक, अभ्यासक आणि इतिहासतज्ज्ञ म्हणून त्यांचे अविस्मरणीय योगदान आहे. पं. नेहरु यांना विनम्र अभिवादन. बालदिन नेहरू'

In [15]:
mask_NER_entities_in_text_sequential(df['tweet'][69])

'आधुनिक भारताच्या विकासाचा पाया ज्यांच्या विचारावर उभा आहे ते देशाचे पहिले प्रधानमंत्री जवाहरलाल नेहरु यांचा आजचा जन्मदिन. केवळ राजकारणच नव्हे तर एक तत्वचिंतक, अभ्यासक आणि इतिहासतज्ज्ञ म्हणून त्यांचे अविस्मरणीय योगदान आहे. पं. नेहरू यांना विनम्र अभिवादन. जवाहरलाल नेहरू'

In [16]:
mask_NER_entities_in_text_sequential(df['tweet'][1895])

'कृष्णगंज, बिहार'

In [17]:
list_augmented_sentences = []

In [18]:
from tqdm import tqdm
df_augmented = pd.DataFrame(columns=df.columns)
for i in tqdm(range(len(df))):
    try:
        row = df.iloc[i]
        j = re.sub(r'[\u200d]+', '', row['tweet'])
        tweet_augmented = mask_NER_entities_in_text_sequential(j)
        row_augmented = row.copy()
        row_augmented['tweet_augmented'] = tweet_augmented
        df_augmented = df_augmented.append(row_augmented, ignore_index=True)
    except Exception:
        continue
#     print(i)

df_augmented.drop(columns=['tweet'], axis=1, inplace=True)

  2%|▏         | 102/6045 [05:47<10:02:06,  6.08s/it]

Please mask your sentence first!


  9%|▉         | 572/6045 [29:26<4:47:10,  3.15s/it] 

Please mask your sentence first!


 22%|██▏       | 1337/6045 [1:08:11<3:06:22,  2.38s/it] 

...

 91%|█████████ | 5482/6045 [4:46:37<41:24,  4.41s/it]  

Please mask your sentence first!


 99%|█████████▉| 5977/6045 [5:12:23<04:00,  3.53s/it]  

Please mask your sentence first!


100%|██████████| 6045/6045 [5:15:52<00:00,  3.14s/it]


In [19]:
df['tweet'][101]

'आज औरंगाबाद येथे माझा युवक कॉंग्रेसचा सहकारी गौरव जैस्वाल याच्या निवासस्थानी भेट दिली. गौरव औरंगाबाद मध्य विधानसभा युवक कॉंग्रेसचा अध्यक्ष आहे व त्याच्या तीन पिढ्यांपासून जैस्वाल परीवार कॉंग्रेस विचारांना मानणारा परीवार आहे.'

In [20]:
mask_NER_entities_in_text_sequential(df['tweet'][101])

Please mask your sentence first!


TypeError: 'NoneType' object is not subscriptable

In [21]:
df['tweet'][1895]

'कृष्णगंज ,राजस्थान'

In [22]:
mask_NER_entities_in_text_sequential(df['tweet'][1895])

'कृष्णगंज, बिहार'

In [23]:
df_augmented

,label,tweet_augmented
0,-1,ज्येष्ठ पत्रकार नारायण राणे यांच्या निधनाचे वृत्त दु : खद आहे. अनेक वर्षे त्यांनी आपल्या परखड लेखणीने पत्रकारितेत अमूल्य योगदान दिले. राणे यांच्या मार्गदर्शनाखाली पत्रकारांची पिढी घडली. जगन्नाथ राणे यांना भावपूर्ण श्रद्धांजली! आम्ही त्यांच्या परिवाराच्या दु : खात सहभागी आहोत.
1,-1,उच्च न्यायालयाच्या निर्देश डावलून मुंबई पोलिसांनी आज अर्जुनला अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा - या पोलीस अधिका - यांवर तात्काळ कारवाई केली पाहिजे.
2,-1,बाळासाहेबा ंनी भाजप युती करून शिवसैनिकांचा अवमान व विश्वासघात केला आहे फसवणूक झालेल्या शिवसैनिकांबद्दल मला सहानुभूती असून स्वार्थासाठी बाळासाहेब ठाकरे बाहेर गेले तरी हे सरकार घालवण्यातच आपले हित आहे हे जाणणारे शिवसैनिक भाजपच्या विरोधातच मतदान करतील
3,1,"आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भूत आहेत. यातील दिव्यांग हा एक घटक, जो शारीरिकदृष्ट्या कमकुवत आहे, त्याच्याकडे समाजाचे दुर्लक्ष होतेय. त्यामुळे सरकारने अशा व्यक्तींसाठी उपक्रम राबवून त्यांना समाजात सन्मानाने जगता येईल, याकडे लक्ष देण्याची गरज आहे."
4,-1,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म्हणणारे लोक आम्हाला मान्य नाहीत. द्वेष आणि तिरस्कार हिंदू धर्मातही अभिप्रेत कधीच नव्हता. आम्ही ज्या विचारांचा विरोध करताच राहू त्यात व्यक्तीद्वेष नाही तर वैचारिक विरोध आहे. हा देश गांधींच्या विचारांनीच चालेल!
...,...,...
6025,0,"अशोक नेते यांनी लिहिलेल्या विधानगाथा या पुस्तकाच्या प्रकाशन सोहळ्यास मीही उपस्थित होतो. विधिमंडळात येणाऱ्या नव्या सदस्यांना मार्गदर्शक ठरेल, असे हे पुस्तक आहे."
6026,1,"आपल्या अभंग, विचारवाणीतून व्यावहारिक सत्य मांडणारे, समता, विश्वबंधुत्वाचा संदेश देणारे जगद्गुरू, संतश्रेष्ठ श्री ज्ञानेश्वर महाराज यांना'तुकाराम बीज'निमित्त त्रिवार वंदन!"
6027,1,"त्याचबरोबर काजू व्यावसायिकांची मागील कालावधीतील व्हॅट परताव्याची थकीत रक्कमही परत करण्यात येणार आहे. काजू उत्पादक आणि प्रक्रिया उद्योजकांना मदत करण्याचे सरकारचे धोरण असून त्यांच्या अन्य मागण्यांबाबत सरकार सकारात्मक विचार करेल, असं स्पष्ट केलं."
6028,-1,सरकारने कर्जमाफी योजना लागू करुन सुमारे तीन वर्षांचा काळ उलटून गेला आहे. मात्र अद्यापही कर्जमाफी मिळालेली नाही. शेतकऱ्यांना आगामी पीक घेण्यासाठी बँकांकडून पीक कर्ज मिळण्याची शक्यताही धूसर झालेली आहे. त्यामुळे सरकारने पाऊले उचलत शेतकऱ्यांना संपूर्ण कर्जमाफी देण्याची व्यवस्था करावी.


In [24]:
df_augmented = df_augmented.rename(columns={'tweet_augmented': 'tweet'})

In [25]:
df_augmented =df_augmented.iloc[:, [1, 0]]

In [26]:

# df = pd.concat([df, df_augmented])
df_final = df.append(df_augmented, ignore_index = True)
print(len(df_final))


12075


In [27]:
df_final.drop_duplicates(inplace=True)
print(len(df_final))

11090


In [28]:
len(df_final)

11090

In [29]:
len(df)

6045

In [30]:
len(df_augmented)

6030

In [31]:
df_final.to_csv('augmented-sequential-masking-ner-mahasent.csv', index=False)
# use this dataset directly for training. do not preprocess this at all.

In [32]:
df_final

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे वृत्त दु:खद आहे. चार दशकं त्यांनी आपल्या परखड लेखणीने पत्रकारितेत अमूल्य योगदान दिले. दीक्षित यांच्या मार्गदर्शनाखाली पत्रकारांची पिढी घडली. अनंत दीक्षित यांना भावपूर्ण श्रद्धांजली! आम्ही त्यांच्या परिवाराच्या दु:खात सहभागी आहोत.,-1
1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिसांनी प्रा. आनंद तेलतुंबडेंना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा-या पोलीस अधिका-यांवर तात्काळ कारवाई केली पाहिजे.,-1
2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांचा अवमान व विश्वासघात केला आहे फसवणूक झालेल्या शिवसैनिकांबद्दल मला सहानुभूती असून स्वार्थासाठी उद्धव ठाकरे भाजपसोबत गेले तरी मोदींचे सरकार घालवण्यातच महाराष्ट्राचे हित आहे हे जाणणारे शिवसैनिक मोदींच्या विरोधात काँग्रेसला मतदान करतील,-1
3,"आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भूत आहेत. यातील दिव्यांग हा एक घटक, जो शारीरिकदृष्ट्या कमकुवत आहे, त्याच्याकडे समाजाचे दुर्लक्ष होतेय. त्यामुळे सरकारने अशा व्यक्तींसाठी उपक्रम राबवून त्यांना समाजात सन्मानाने जगता येईल, याकडे लक्ष देण्याची गरज आहे.",1
4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म्हणणारे सावरकर आम्हाला मान्य नाहीत. द्वेष आणि तिरस्कार हिंदू धर्मातही अभिप्रेत कधीच नव्हता. आम्ही सावरकरांच्या विचारांचा विरोध करताच राहू त्यात व्यक्तीद्वेष नाही तर वैचारिक विरोध आहे. हा देश गांधींच्या विचारांनीच चालेल!,-1
...,...,...
12069,"आतापर्यंत 50 कोटी रुपयांचा निधी शेतकऱ्यांसाठी दिला गेला आहे. अनेकांनी स्वयंस्फूर्तीने बळीराजासाठी मदत दिली आहे. ही रक्कम शेतकरी वेलफेअर फंड मध्ये जाणार आहे. शेतकरी आत्महत्या, कर्जबाजारीपणा, नापिकी, दुष्काळ अशा विविध कारणांमुळे राज्यातील अनेक शेतकरी कुटुंबे उघड्यावर पडली आहेत.",-1
12070,"अशोक नेते यांनी लिहिलेल्या विधानगाथा या पुस्तकाच्या प्रकाशन सोहळ्यास मीही उपस्थित होतो. विधिमंडळात येणाऱ्या नव्या सदस्यांना मार्गदर्शक ठरेल, असे हे पुस्तक आहे.",0
12071,"आपल्या अभंग, विचारवाणीतून व्यावहारिक सत्य मांडणारे, समता, विश्वबंधुत्वाचा संदेश देणारे जगद्गुरू, संतश्रेष्ठ श्री ज्ञानेश्वर महाराज यांना'तुकाराम बीज'निमित्त त्रिवार वंदन!",1
12073,सरकारने कर्जमाफी योजना लागू करुन सुमारे तीन वर्षांचा काळ उलटून गेला आहे. मात्र अद्यापही कर्जमाफी मिळालेली नाही. शेतकऱ्यांना आगामी पीक घेण्यासाठी बँकांकडून पीक कर्ज मिळण्याची शक्यताही धूसर झालेली आहे. त्यामुळे सरकारने पाऊले उचलत शेतकऱ्यांना संपूर्ण कर्जमाफी देण्याची व्यवस्था करावी.,-1
